**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random as rd
from collections import deque

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
    
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Function ```act```: Given a state in a certain environment, this function will determine what will the next action be, based on a certain policy. However, a learned action will not take into consideration the probability of having a random action. By doing exploration it will check all the possible options and then choose the best one. This is where $\epsilon$ has a role. It works as follows: A value is generated randomly, if it is less than $\epsilon$, a random action is chosen, otherwise, the optimal max Q value will be chosen.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` : Represents the current position of the rat on the board.

```board``` : represents all the possible cells, this translates to the fact that ```board``` is a representation of all the possible states

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return rd.randint(0,3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.5/14.0. Average score (-1.5)
Win/lose count 5.5/8.0. Average score (-2.0)
Win/lose count 10.5/10.0. Average score (-1.1666666666666667)
Win/lose count 11.5/10.0. Average score (-0.5)
Win/lose count 12.0/20.0. Average score (-2.0)
Win/lose count 10.0/21.0. Average score (-3.5)
Win/lose count 12.5/7.0. Average score (-2.2142857142857144)
Win/lose count 14.5/17.0. Average score (-2.25)
Win/lose count 6.5/8.0. Average score (-2.1666666666666665)
Win/lose count 11.0/11.0. Average score (-1.95)
Win/lose count 3.5/18.0. Average score (-3.090909090909091)
Win/lose count 11.0/15.0. Average score (-3.1666666666666665)
Win/lose count 7.5/8.0. Average score (-2.9615384615384617)
Win/lose count 12.0/14.0. Average score (-2.892857142857143)
Win/lose count 10.0/7.0. Average score (-2.5)
Win/lose count 6.5/10.0. Average score (-2.5625)
Win/lose count 15.0/16.0. Average score (-2.4705882352941178)
Win/lose count 8.0/9.0. Average score (-2.388888888888889)
Win/lose count 8.5/9.0. Avera

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**ANSWER**

The bellman equation is given as follows:
\begin{equation*}
Q^{\pi}(s,a)  = E_{\pi}[\Sigma^{\infty}_{t=0} \gamma^t r_t(s_t, \pi(s_t),s_{t+1}) | s_t = s,a_t = a] \\
\end{equation*}

Using the Markov Property, we can take the first element of the sum outside, we will be left with:

\begin{align*}
Q^{\pi}(s_0,a_0) & = E_{\pi}[r_0(s_0,a_0,s_1)+\Sigma^{\infty}_{t=1} \gamma^t r_t (s_t,\pi(s_t),s_{t+1}) | s_0,a_0 ] \\
& = E_{\pi}[r_0(s_0,a_0,s_1) + \gamma E_{\pi} \{ \Sigma_{t=1}^{\infty} \gamma^{t-1} r_t (s_t,\pi(s_t),s_{t+1} | s_1,a_1 \} s_0,a_0 ] \\
& = E_{\pi} [r_0 (s_0,a_0,s_1)+\gamma Q^{\pi}(s_1,a_1)] \\
& = E_{(s',a') \sim p(.|s,a)} [r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

Given an ultimate policy $\pi^*$ , the optimal Q function can be written as:

\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> 
\end{equation*}

Using the previous equation that we derived in the first step:

\begin{equation*}
Q^*(s,a)= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \> 
\end{equation*}

\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')] \> 
\end{equation*}

\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}



Given the loss function $\mathcal{L}(\theta)= [Q^*(s,a,\theta) - Q(s,a,\theta)]^2$

$$\mathcal{L}(\theta) = [E_{s'\sim \pi^*(.|s,a,s')}[r(s,a,s')+\gamma\max_{a'}Q^{*}(s',a,\theta)] - Q(s,a,\theta)]^2$$
$$ {\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}} $$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)

    def random_access(self): 
        return self.memory[rd.randint(0,len(self.memory)-1)]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s = np.reshape(s,[1,5,5,self.n_state])
        pred = self.model.predict (s) 
        return np.argmax(pred[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
 # Two steps: first memorize the states, second learn from the pool

        s_ = np.reshape(s_,[1,5,5,self.n_state])
        n_s_ = np.reshape(n_s_,[1,5,5,self.n_state])
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
    
        
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            target = r_
            s_, n_s_,a_, r_, game_over_ = self.memory.random_access()
            input_states[i]= s_
            
            if game_over_:
                target= r_

            else:               
                target = (r_ + self.discount * np.amax(self.model.predict(n_s_)[0]))
                
            target_f = self.model.predict(s_)
            target_f[0][a_] = target
            self.model.fit(s_, target_f, epochs=1, verbose=0)
            
            target_q[i]=target_f
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model = 'model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(128,input_shape=(50,),init='uniform',activation='relu'))
        model.add(Dense(64,init='uniform',activation='relu'))
        model.add(Dense(32,init='uniform',activation='relu'))
        model.add(Dense(16,activation='relu'))
        model.add(Dense(8,activation='relu'))
        model.add(Dense(4,init='uniform',activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_shape=(50,), activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:84: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:85: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:88: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="linear", kernel_initializer="uniform")`


Epoch 000/020 | Loss 0.0116 | Win/lose count 5.5/8.0 (-2.5)
Epoch 001/020 | Loss 0.0245 | Win/lose count 7.5/3.0 (4.5)
Epoch 002/020 | Loss 0.0281 | Win/lose count 11.5/20.0 (-8.5)
Epoch 003/020 | Loss 0.0055 | Win/lose count 7.5/15.0 (-7.5)
Epoch 004/020 | Loss 0.0180 | Win/lose count 4.5/9.0 (-4.5)
Epoch 005/020 | Loss 0.0164 | Win/lose count 2.0/5.0 (-3.0)
Epoch 006/020 | Loss 0.0793 | Win/lose count 7.5/9.0 (-1.5)
Epoch 007/020 | Loss 0.0244 | Win/lose count 4.0/6.0 (-2.0)
Epoch 008/020 | Loss 0.0473 | Win/lose count 5.5/4.0 (1.5)
Epoch 009/020 | Loss 0.0097 | Win/lose count 2.0/5.0 (-3.0)
Epoch 010/020 | Loss 0.0185 | Win/lose count 6.5/5.0 (1.5)
Epoch 011/020 | Loss 0.0214 | Win/lose count 7.0/7.0 (0.0)
Epoch 012/020 | Loss 0.0142 | Win/lose count 2.0/5.0 (-3.0)
Epoch 013/020 | Loss 0.0091 | Win/lose count 4.0/1.0 (3.0)
Epoch 014/020 | Loss 0.0294 | Win/lose count 2.0/2.0 (0.0)
Epoch 015/020 | Loss 0.0063 | Win/lose count 4.5/5.0 (-0.5)
Epoch 016/020 | Loss 0.0122 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        
    
        
        model = Sequential()
        model.add(Conv2D(32, (2,2), input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(32, (2,2), activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0123 | Win/lose count 3.5/8.0 (-4.5)
Epoch 001/020 | Loss 0.0040 | Win/lose count 6.5/2.0 (4.5)
Epoch 002/020 | Loss 0.0125 | Win/lose count 4.0/3.0 (1.0)
Epoch 003/020 | Loss 0.0227 | Win/lose count 10.0/7.0 (3.0)
Epoch 004/020 | Loss 0.1266 | Win/lose count 16.5/2.0 (14.5)
Epoch 005/020 | Loss 0.0734 | Win/lose count 18.5/5.0 (13.5)
Epoch 006/020 | Loss 0.2651 | Win/lose count 12.5/6.0 (6.5)
Epoch 007/020 | Loss 0.2538 | Win/lose count 18.0/5.0 (13.0)
Epoch 008/020 | Loss 0.3786 | Win/lose count 18.0/5.0 (13.0)
Epoch 009/020 | Loss 0.3707 | Win/lose count 17.0/5.0 (12.0)
Epoch 010/020 | Loss 0.6231 | Win/lose count 15.5/1.0 (14.5)
Epoch 011/020 | Loss 0.2422 | Win/lose count 20.5/9.0 (11.5)
Epoch 012/020 | Loss 0.3343 | Win/lose count 23.0/4.0 (19.0)
Epoch 013/020 | Loss 0.4475 | Win/lose count 9.5/2.0 (7.5)
Epoch 014/020 | Loss 0.5429 | Win/lose count 24.0/5.0 (19.0)
Epoch 015/020 | Loss 0.3250 | Win/lose count 15.0/2.0 (13.0)
Epoch 016/020 | Loss 0.1908 | Win

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [16]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_shape=(50,), activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:84: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:85: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:88: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="linear", kernel_initializer="uniform")`


Test of the CNN
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/0. Average score (0.25)
Win/lose count 1.0/1.0. Average score (0.16666666666666666)
Win/lose count 1.0/1.0. Average score (0.125)
Win/lose count 0.5/1.0. Average score (0.0)
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0.5/0. Average score (0.0625)
Win/lose count 0/0. Average score (0.05555555555555555)
Win/lose count 1.0/0. Average score (0.15)
Win/lose count 1.5/1.0. Average score (0.18181818181818182)
Win/lose count 0/0. Average score (0.16666666666666666)
Win/lose count 1.0/0. Average score (0.23076923076923078)
Win/lose count 0.5/3.0. Average score (0.03571428571428571)
Win/lose count 3.0/2.0. Average score (0.1)
Win/lose count 0.5/0. Average score (0.125)
Win/lose count 1.0/0. Average score (0.17647058823529413)
Win/lose count 0.5/3.0. Average score (0.027777777777777776)
Win/lose count 0/2.0. Average score (-0.07894736842105263)
Win/lose count 0.5/0. A

In [17]:
HTML(display_videos('cnn_test10.mp4'))

In [18]:
HTML(display_videos('fc_test10.mp4'))

- First we observe that the model'sexploration rate is low. In fact, only the current board is explored. Another observation is the performance of both CNN and FC models, where we notice that the CNN model outperforms the FC model.

- Changing the temperature results in better scores. However, this approach is biased and cannot be considered as a good solution since it is playing with the binomial law.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # Epsilon-greedy 
            agent.set_epsilon(np.float(agent.get_epsilon())*0.995)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4       
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        self.board = np.zeros([grid_size,grid_size])
        self.position = np.zeros([grid_size,grid_size])
        self.malus_position = np.zeros([grid_size,grid_size])
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def get_t(self):
        return self.t
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
             
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]

        self.malus_position[self.x, self.y] = 0.01
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
       
        self.malus_position[self.x, self.y] = 0

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0020 | Win/lose count 1.0/5.86999999999996 (-4.86999999999996)
Epoch 001/020 | Loss 0.0029 | Win/lose count 2.0/3.90999999999996 (-1.9099999999999602)
Epoch 002/020 | Loss 0.0006 | Win/lose count 0.5/1.9300000000000015 (-1.4300000000000015)
Epoch 003/020 | Loss 0.0023 | Win/lose count 5.49/2.799999999999984 (2.6900000000000164)
Epoch 004/020 | Loss 0.0085 | Win/lose count 2.0/3.9199999999999733 (-1.9199999999999733)
Epoch 005/020 | Loss 0.0064 | Win/lose count 3.4400000000000004/3.899999999999963 (-0.45999999999996266)
Epoch 006/020 | Loss 0.0040 | Win/lose count 4.420000000000001/2.8999999999999813 (1.5200000000000196)
Epoch 007/020 | Loss 0.0053 | Win/lose count 5.49/3.7999999999999625 (1.6900000000000377)
Epoch 008/020 | Loss 0.0147 | Win/lose count 4.410000000000001/2.919999999999981 (1.4900000000000202)
Epoch 009/020 | Loss 0.0020 | Win/lose count 10.850000000000001/4.719999999999965 (6.130000000000036)
Epoch 010/020 | Loss 0.0047 | Win/lose count 2.49/1.910

In [27]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 10.780000000000003/1.7900000000000014. Average score (8.990000000000002)
Win/lose count 8.330000000000002/1.8200000000000014. Average score (7.750000000000001)
Win/lose count 11.770000000000003/2.769999999999984. Average score (8.166666666666673)
Win/lose count 1.96/1.9700000000000015. Average score (6.122500000000004)
Win/lose count 1.47/1.9800000000000015. Average score (4.796000000000003)
Win/lose count 5.3900000000000015/3.8899999999999633. Average score (4.246666666666676)
Win/lose count 1.47/1.9800000000000015. Average score (3.5671428571428647)
Win/lose count 10.310000000000002/2.7999999999999834. Average score (4.060000000000009)
Win/lose count 6.860000000000002/1.8600000000000014. Average score (4.164444444444452)
Win/lose count 5.3900000000000015/1.9000000000000015. Average score (4.0970000000000075)
Win/lose count 6.860000000000002/2.8399999999999825. Average score (4.090000000000009)
Win/lose count 3.920000000000001/1.9200000000000015. Average score (3.915833

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***